In [4]:
import torch
import walnut

In [5]:
X = walnut.randn((10, 3, 3))
W = walnut.ones((3, 3))
B = walnut.zeros((3, 3))

eps = 1e-7

t_x = torch.nn.Parameter(torch.from_numpy(X.data).float(), requires_grad=True)
t_w = torch.nn.Parameter(torch.from_numpy(W.data).float(), requires_grad=True)
t_b = torch.nn.Parameter(torch.from_numpy(B.data).float(), requires_grad=True)

# Forward
### NumpyNN

In [6]:
import walnut.nn as nn

ln = nn.modules.Layernorm(input_shape=X.shape[1:], eps=eps)
ln.compile()
ln.w = W
ln.b = B
ln(X)[0]

tnsor([[-0.9451,  0.2960, -0.3816],
       [ 0.4564,  0.3498, -1.5783],
       [-0.7530,  0.4972,  2.0586]], dtype=float32)

### Torch

In [7]:
ln_t = torch.nn.LayerNorm(X.shape[-2:], eps=eps)
ln_t.weight = t_w
ln_t.bias = t_b
t_out = ln_t(t_x)
t_out[0]

tensor([[-0.9451,  0.2960, -0.3816],
        [ 0.4564,  0.3498, -1.5783],
        [-0.7530,  0.4972,  2.0586]], grad_fn=<SelectBackward0>)

# Backward

In [8]:
dy = walnut.randn(X.shape).data
t_dy = torch.nn.Parameter(torch.from_numpy(dy).float())

In [9]:
dy = walnut.randn(X.shape).data
t_dy = torch.from_numpy(dy)
_ = ln.backward(dy)
t_out.backward(t_dy)

### Beta

In [10]:
ln.b.grad

array([[-0.9337,  4.9937,  1.6646],
       [ 2.8507,  1.4694,  0.8327],
       [-1.6684,  1.3044, -1.5483]], dtype=float32)

In [11]:
ln_t.bias.grad

tensor([[-0.9337,  4.9937,  1.6646],
        [ 2.8507,  1.4694,  0.8327],
        [-1.6684,  1.3044, -1.5483]])

### Gamma

In [12]:
ln.w.grad

array([[ 2.6746,  6.6361, -1.2104],
       [-1.0429, -1.1509, -3.7503],
       [ 4.4661,  7.3159, -2.4227]], dtype=float32)

In [13]:
ln_t.weight.grad

tensor([[ 2.6746,  6.6361, -1.2104],
        [-1.0429, -1.1509, -3.7503],
        [ 4.4661,  7.3159, -2.4227]])

### X

In [14]:
ln.x.grad[0]

array([[-1.1761,  0.6990, -1.1435],
       [ 1.0920,  0.0342,  1.3114],
       [-0.5299,  0.0021, -0.2892]], dtype=float32)

In [15]:
t_x.grad[0]

tensor([[-1.1761,  0.6990, -1.1435],
        [ 1.0920,  0.0342,  1.3114],
        [-0.5299,  0.0021, -0.2892]])